In [1]:
# Import libraries
import torch
from torch import tensor
from torch import nn 
from torch import sigmoid
from torch import atan
from torch import tanh
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random

seed = 7777
random.seed(seed) 
torch.manual_seed(seed)

In [ ]:
lpy_min = 50 #mm
lpy_max = 2050 #mm
lpx_min = 50 #mm
lpx_max = 650 #mm
wp_min = 25 # mm
wp_max = 325 # mm
a_min = 0 # mm
a_max = 200 # mm
p_min = 0 # mm
p_max = 200 # mm
ls_min = 50 # mm
ls_max = 450 # mm
ws_min = 25 # mm
ws_max = 225 # mm
# Ip_min = 50 # A rms
# Ip_max = 200 # A rms
# Np_min = 4 #turn
# Np_max = 10 #turn
# Ns_min = 4 #turn
# Ns_max = 10 #turn

In [2]:
class Model(nn.Module): # Design your model using class
    def __init__(self):    
        super(Model, self).__init__() #In the constructor, we instantiate nn.Linear module.
        self.linear1 = nn.Linear(12, 100, bias=True).cuda() # nn.Linear(<input size> ,<output size>)
        self.linear2 = nn.Linear(100, 100, bias=True).cuda()
        self.linear3 = nn.Linear(100, 42, bias=True).cuda()

    def forward(self, x):
        x = atan(self.linear1(x))
        x = atan(self.linear2(x))
        x = atan(self.linear2(x))
        y_pred = self.linear3(x)
        return y_pred

In [3]:
model = Model() # our model
model.load_state_dict(torch.load('saved_model_state.pt'))

for param in model.parameters():
    param.requires_grad = False

In [4]:
def generate_noise(a, b):
    low,high = 0, 1  # range of uniform distribution

    return torch.distributions.uniform.Uniform(low,high).sample([a,b]).cuda()

In [25]:
class Generator(nn.Module):

    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), 128).cuda()
        self.dense_layer1 = nn.Linear(128, 256).cuda()
        self.dense_layer2 = nn.Linear(256, 512).cuda()
        self.dense_layer3 = nn.Linear(512, 1024).cuda()
        self.dense_layer4 = nn.Linear(1024, 512).cuda()
        self.dense_layer5 = nn.Linear(512, 256).cuda()
        self.dense_layer6 = nn.Linear(256, 128).cuda()
        self.dense_layer7 = nn.Linear(128, int(input_length)).cuda()
        self.activation = nn.Sigmoid()
        

    def forward(self, x):
        x = atan(self.dense_layer(x))
        x = atan(self.dense_layer1(x))
        x = atan(self.dense_layer2(x))
        x = atan(self.dense_layer3(x))
        x = atan(self.dense_layer4(x))
        x = atan(self.dense_layer5(x))
        x = atan(self.dense_layer6(x))
        x = self.activation(self.dense_layer7(x))
        return x

In [26]:
generator = Generator(7)
generator.requires_grad_= True

In [27]:
def scaler(value, min , max):
    return value * (max - min) + min

In [28]:
@torch.jit.script
def linspace(start: torch.Tensor, stop: torch.Tensor, num: int):
    """
    Creates a tensor of shape [num, *start.shape] whose values are evenly spaced from start to end, inclusive.
    Replicates but the multi-dimensional bahaviour of numpy.linspace in PyTorch.
    """
    # create a tensor of 'num' steps from 0 to 1
    steps = torch.arange(num, dtype=torch.float32, device=start.device) / (num - 1)
    
    # reshape the 'steps' tensor to [-1, *([1]*start.ndim)] to allow for broadcastings
    # - using 'steps.reshape([-1, *([1]*start.ndim)])' would be nice here but torchscript
    #   "cannot statically infer the expected size of a list in this contex", hence the code below
    for i in range(start.ndim):
        steps = steps.unsqueeze(-1)
    
    # the output starts at 'start' and increments until 'stop' in each dimension
    out = start[None] + steps*(stop - start)[None]
    
    return out

In [33]:
def scale_and_extract(arr):
    print(arr[0])
    scaler_min = torch.tensor([50., 50., 25., 0., 0., 50., 25.], requires_grad=True, device="cuda")
    scaler_max = torch.tensor([2050., 650., 325., 200., 200., 450., 225.], requires_grad=True, device="cuda")
    arr = scaler(arr, scaler_min, scaler_max)
    print(arr[0])
    y0_i = (arr[:,0] + arr[:,2]) / 2
    y1_i = (3/2 * arr[:,0]) + (5/2 * arr[:,2]) + (2 * arr[:,3]) + (arr[:,4])

    ys_min = (y1_i)
    ys_max = (y1_i + (y1_i - y0_i) / 2)
    
    ys = linspace(ys_min, ys_max, num = 5)
    ys = torch.transpose(ys, 0, 1)

    return torch.cat([arr, ys], axis=1)


In [30]:
TRAINING = 50000
learning_rate = 3e-4
optimizer = torch.optim.Adam(generator.parameters(), lr = learning_rate)

In [35]:
for i in range(TRAINING):
    
    # zero out the parameter gradients
    # self.optimizer.zero_grad() is slow
    for param in generator.parameters():
        param.grad = None

    noise = generate_noise(10000, 7)
    print(noise)
    GP = generator(noise)
    
    scaled_array = scale_and_extract(GP)
    print(scaled_array[0])
    break
    MP = model(scaled_array)
    
    kdiff = MP[:, 0] - MP[:, 5]
    kdiff = abs(kdiff / MP[:, 0])
    kdiff = torch.mean(kdiff)
    
    kdiff.backward()
    optimizer.step()
    
    if i % 2500 == 0:
        print(f"Epoch: {i:4d} | Coupling: {100 * kdiff:.3f}%")


tensor([[0.7594, 0.7356, 0.2556,  ..., 0.2930, 0.9439, 0.6944],
        [0.3204, 0.8819, 0.3080,  ..., 0.4136, 0.0314, 0.1652],
        [0.3942, 0.8906, 0.9813,  ..., 0.0291, 0.8477, 0.6556],
        ...,
        [0.9581, 0.8677, 0.5728,  ..., 0.0173, 0.4631, 0.1861],
        [0.4225, 0.7138, 0.8621,  ..., 0.6353, 0.8093, 0.3532],
        [0.5751, 0.7198, 0.5447,  ..., 0.8842, 0.8882, 0.6736]],
       device='cuda:0')
tensor([9.9997e-01, 7.3271e-01, 6.1664e-01, 5.8002e-01, 6.7957e-06, 9.9999e-01,
        9.9998e-01], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.0499e+03, 4.8963e+02, 2.0999e+02, 1.1600e+02, 1.3591e-03, 4.5000e+02,
        2.2500e+02], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.0499e+03, 4.8963e+02, 2.0999e+02, 1.1600e+02, 1.3591e-03, 4.5000e+02,
        2.2500e+02, 3.8319e+03, 4.1696e+03, 4.5074e+03, 4.8451e+03, 5.1829e+03],
       device='cuda:0', grad_fn=<SelectBackward0>)
